### Подведите результаты эксперимента в экселе по следующим данным: ab_stats.csv - Google Диск: Стат значимо ли отличается ARPPU в двух группах ? Какие рекомендации дадите менеджеру?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from statsmodels.stats import proportion

In [2]:
df = pd.read_csv('ab_stats.csv')
df

,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
...,...,...,...,...,...
23647,0.0,0,0,B,10.207152
23648,0.0,0,0,B,5.148761
23649,0.0,0,0,B,4.809019
23650,0.0,0,0,B,8.127488


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23652 entries, 0 to 23651
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        23652 non-null  float64
 1   num_purchases  23652 non-null  int64  
 2   purchase       23652 non-null  int64  
 3   ab_group       23652 non-null  object 
 4   av_site visit  23652 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 924.0+ KB


In [4]:
df.describe()

,revenue,num_purchases,purchase,av_site visit
count,23652.000000,23652.000000,23652.000000,23652.000000
mean,0.324689,0.043590,0.020717,7.013112
std,9.557730,1.079403,0.142438,3.154584
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.173787
50%,0.000000,0.000000,0.000000,7.007936
75%,0.000000,0.000000,0.000000,8.864119
max,1303.609284,152.000000,1.000000,22.446822


Минимальное значение av_site visit отрицательное, следоватьельно есть ошибочные данные

In [6]:
df[df['av_site visit'] < 0].count()


revenue          468
num_purchases    468
purchase         468
ab_group         468
av_site visit    468
dtype: int64

468*100/23652=1.98%. Отрицательных значений от общего количества значений меньще 5%, можно их исключить

In [8]:
df2 = df[df['av_site visit'] >= 0]
df2

,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814
...,...,...,...,...,...
23647,0.0,0,0,B,10.207152
23648,0.0,0,0,B,5.148761
23649,0.0,0,0,B,4.809019
23650,0.0,0,0,B,8.127488


In [10]:
df2.describe()

,revenue,num_purchases,purchase,av_site visit
count,23184.000000,23184.000000,23184.000000,23184.000000
mean,0.326511,0.043780,0.020704,7.188254
std,9.645839,1.089462,0.142394,2.924011
min,0.000000,0.000000,0.000000,0.008679
25%,0.000000,0.000000,0.000000,5.314379
50%,0.000000,0.000000,0.000000,7.068656
75%,0.000000,0.000000,0.000000,8.910900
max,1303.609284,152.000000,1.000000,22.446822


In [50]:
df2['revenue'].sum()

7569.839526999998

Убираем результаты, в которых покупка не совершалась. Для ARPPU они не требуются.

In [54]:
df3=df2[df2['purchase'] > 0]
df3

,revenue,num_purchases,purchase,ab_group,av_site visit
45,1.885595,1,1,A,7.654627
54,1.002159,1,1,A,6.392489
82,2.990000,1,1,A,8.596604
104,49.990000,1,1,A,8.885633
110,22.093757,4,1,A,8.708759
...,...,...,...,...,...
23426,2.489611,1,1,B,9.015714
23493,74.950000,5,1,B,5.881950
23495,3.667866,1,1,B,7.450014
23584,19.990000,1,1,B,9.813696


In [58]:
df3.describe()

,revenue,num_purchases,purchase,av_site visit
count,480.000000,480.000000,480.0,480.000000
mean,15.770499,2.114583,1.0,6.983437
std,65.261464,7.284080,0.0,2.851683
min,0.259373,1.000000,1.0,0.021794
25%,2.303961,1.000000,1.0,5.190029
50%,4.094598,1.000000,1.0,6.987301
75%,12.021209,2.000000,1.0,8.725514
max,1303.609284,152.000000,1.0,16.955948


In [52]:
df3['revenue'].sum()

7569.839526999998

Добавляем в таблицу столбец ARPPU.

In [59]:
df4=df3.assign(arppu=df3['revenue']/df2['num_purchases'])
df4

,revenue,num_purchases,purchase,ab_group,av_site visit,arppu
45,1.885595,1,1,A,7.654627,1.885595
54,1.002159,1,1,A,6.392489,1.002159
82,2.990000,1,1,A,8.596604,2.990000
104,49.990000,1,1,A,8.885633,49.990000
110,22.093757,4,1,A,8.708759,5.523439
...,...,...,...,...,...,...
23426,2.489611,1,1,B,9.015714,2.489611
23493,74.950000,5,1,B,5.881950,14.990000
23495,3.667866,1,1,B,7.450014,3.667866
23584,19.990000,1,1,B,9.813696,19.990000


In [60]:
df_a = df4[df4['ab_group'] == 'A']
df_b = df4[df4['ab_group'] == 'B']

In [61]:
df_a.describe()

,revenue,num_purchases,purchase,av_site visit,arppu
count,256.000000,256.000000,256.0,256.000000,256.000000
mean,18.698469,2.343750,1.0,6.753074,6.548581
std,87.527589,9.723582,0.0,2.043753,11.011433
min,0.259689,1.000000,1.0,0.996753,0.259689
25%,2.318301,1.000000,1.0,5.281247,1.990000
50%,4.283160,1.000000,1.0,6.871241,3.533764
75%,11.923141,2.000000,1.0,8.257506,6.912193
max,1303.609284,152.000000,1.0,12.504262,136.195744


In [62]:
df_b.describe()

,revenue,num_purchases,purchase,av_site visit,arppu
count,224.000000,224.000000,224.0,224.000000,224.000000
mean,12.424248,1.852679,1.0,7.246710,6.564638
std,19.153743,2.392194,0.0,3.544114,8.913380
min,0.259373,1.000000,1.0,0.021794,0.259373
25%,2.286042,1.000000,1.0,4.665967,1.990000
50%,3.980000,1.000000,1.0,7.370188,2.990000
75%,12.462425,2.000000,1.0,9.814082,6.800803
max,113.830000,25.000000,1.0,16.955948,61.683456


In [64]:
all_a = df_a.arppu.value_counts().sum()
arppu_a = df_a['arppu'].mean()
all_a, arppu_a

(256, 6.5485810247464356)

In [65]:
all_b = df_b.arppu.value_counts().sum()
arppu_b = df_b['arppu'].mean()
all_b, arppu_b

(224, 6.564637633463384)

Проверяем распределение на нормальность


In [66]:
st.shapiro(df_a['arppu'])

ShapiroResult(statistic=0.42073702812194824, pvalue=6.399404907942076e-28)

In [67]:
st.shapiro(df_b['arppu'])

ShapiroResult(statistic=0.5905328989028931, pvalue=9.715646249670535e-23)

Значания p-value меньше 0,05. Следовательно распределнеие не нормальное

In [68]:
st.mannwhitneyu(df_a['arppu'], df_b['arppu'])

MannwhitneyuResult(statistic=28853.5, pvalue=0.9049132020870465)

P-value>0,05 следовательно значемых различий по ARPPU нет. 

## Вывод: Необходимо оставлять нулевую теорию.